In [2]:
#clone YOLOv5 and 
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
# %pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

/home/sergey.zemskov/people-detection-omsk/yolov5
Note: you may need to restart the kernel to use updated packages.


/home/sergey.zemskov/anaconda3/envs/torch_cuda117/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete. Using torch 1.13.0+cu117 (Quadro T1000 with Max-Q Design)


In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

IMG_SIZE = 864

In [3]:
!nvidia-smi

Thu Nov 10 01:31:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.75       Driver Version: 517.40       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro T1000 wi...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P8     3W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import gc
torch.cuda.empty_cache()
n = gc.collect()
print("Number of unreachable objects collected by GC:", n)

Number of unreachable objects collected by GC: 19


In [5]:
a = torch.zeros(300000000, dtype=torch.int8)
a = a.cuda()
del a
torch.cuda.empty_cache()

In [6]:
!python train.py --img {IMG_SIZE} --batch 2 --epochs 350 --data ../yolo_config.yaml --hyp ../hyp.yaml --weights yolov5m.pt --optimizer 'AdamW' --cache ram

train: weights=yolov5m.pt, cfg=, data=../yolo_config.yaml, hyp=../hyp.yaml, epochs=350, batch_size=2, imgsz=864, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 8 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

hyperparameters: lr0=0.0008, lrf=0.05, momentum=0.937, weight_decay=0.0005, warmup_epochs=20.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0

In [10]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10707), started 0:00:05 ago. (Use '!kill 10707' to kill it.)

In [7]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.1 --source ../val/images --line-thickness 1

detect: weights=['runs/train/exp13/weights/best.pt'], source=../val/images, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/260 /home/sergey.zemskov/people-detection-omsk/val/images/7503.JPG: 512x864 (no detections), 99.5ms
image 2/260 /home/sergey.zemskov/people-detection-omsk/val/images/7506.JPG: 512x864 (no detections), 99.2ms
image 3/260 /home/sergey.zemskov/people-detection-omsk/val/images/7507.JPG: 512x864 (no detections), 96.4ms
image

In [8]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.1 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/best.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 1 person, 102.0ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 91.0ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 1 person, 88.0ms
image 4/3435 /home/sergey.zemskov/people-detect

In [11]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.2 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/best.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 105.5ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 96.3ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 (no detections), 106.8ms
image 4/3435 /home/sergey.zemsko

In [12]:
!python detect.py --weights runs/train/exp13/weights/last.pt --img {IMG_SIZE} --conf 0.25 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/last.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 104.0ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 92.9ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 (no detections), 90.6ms
image 4/3435 /home/sergey.zemsko

In [13]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.25 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/best.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 86.9ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 81.9ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 (no detections), 82.0ms
image 4/3435 /home/sergey.zemskov

In [14]:
!python detect.py --weights runs/train/exp13/weights/last.pt --img {IMG_SIZE} --conf 0.2 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/last.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 95.4ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 78.6ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 1 person, 77.9ms
image 4/3435 /home/sergey.zemskov/people-

In [15]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.15 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/best.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.15, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 91.1ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 71.6ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 1 person, 71.9ms
image 4/3435 /home/sergey.zemskov/people

In [16]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.17 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/best.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.17, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 97.6ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 93.2ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 1 person, 92.3ms
image 4/3435 /home/sergey.zemskov/people

In [17]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.15 --iou-thres 0.3 --source ../test --save-txt --nosave --line-thickness 1 

detect: weights=['runs/train/exp13/weights/best.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.15, iou_thres=0.3, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 108.4ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 141.5ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 1 person, 101.6ms
image 4/3435 /home/sergey.zemskov/peop

In [4]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.14 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/best.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.14, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 105.5ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 91.7ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 1 person, 89.7ms
image 4/3435 /home/sergey.zemskov/peopl

In [5]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img {IMG_SIZE} --conf 0.14 --iou-thres 0.3 --source ../test --save-txt --nosave --line-thickness 1

detect: weights=['runs/train/exp13/weights/best.pt'], source=../test, data=data/coco128.yaml, imgsz=[864, 864], conf_thres=0.14, iou_thres=0.3, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-227-g78ed31c Python-3.9.13 torch-1.13.0+cu117 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/3435 /home/sergey.zemskov/people-detection-omsk/test/1.JPG: 512x864 (no detections), 87.5ms
image 2/3435 /home/sergey.zemskov/people-detection-omsk/test/10.JPG: 512x864 (no detections), 79.8ms
image 3/3435 /home/sergey.zemskov/people-detection-omsk/test/100.JPG: 512x864 1 person, 77.2ms
image 4/3435 /home/sergey.zemskov/people-